# Amazon SageMaker Autopilot Candidate Definition Notebook

This notebook was automatically generated by the AutoML job **automl-woobin**.
This notebook allows you to customize the candidate definitions and execute the SageMaker Autopilot workflow.

The dataset has **21** columns and the column named **Churn?** is used as
the target column. This is being treated as a **BinaryClassification** problem. The dataset also has **2** classes.
This notebook will build a **[BinaryClassification](https://en.wikipedia.org/wiki/Binary_classification)** model that
**maximizes** the "**F1**" quality metric of the trained models.
The "**F1**" metric applies for binary classification with a positive and negative class. It mixes between precision and recall, and is recommended in cases where there are more negative examples compared to positive examples.

As part of the AutoML job, the input dataset has been randomly split into two pieces, one for **training** and one for
**validation**. This notebook helps you inspect and modify the data transformation approaches proposed by Amazon SageMaker Autopilot. You can interactively
train the data transformation models and use them to transform the data. Finally, you can execute a multiple algorithm hyperparameter optimization (multi-algo HPO)
job that helps you find the best model for your dataset by jointly optimizing the data transformations and machine learning algorithms.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>
Look for sections like this for recommended settings that you can change.
</div>


---

## Contents

1. [Sagemaker Setup](#Sagemaker-Setup)
    1. [Downloading Generated Candidates](#Downloading-Generated-Modules)
    1. [SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration](#SageMaker-Autopilot-Job-and-Amazon-Simple-Storage-Service-(Amazon-S3)-Configuration)
1. [Candidate Pipelines](#Candidate-Pipelines)
    1. [Generated Candidates](#Generated-Candidates)
    1. [Selected Candidates](#Selected-Candidates)
1. [Executing the Candidate Pipelines](#Executing-the-Candidate-Pipelines)
    1. [Run Data Transformation Steps](#Run-Data-Transformation-Steps)
    1. [Multi Algorithm Hyperparameter Tuning](#Multi-Algorithm-Hyperparameter-Tuning)
1. [Model Selection and Deployment](#Model-Selection-and-Deployment)
    1. [Tuning Job Result Overview](#Tuning-Job-Result-Overview)
    1. [Model Deployment](#Model-Deployment)

---

## Sagemaker Setup

Before you launch the SageMaker Autopilot jobs, we'll setup the environment for Amazon SageMaker
- Check environment & dependencies.
- Create a few helper objects/function to organize input/output data and SageMaker sessions.

**Minimal Environment Requirements**

- Jupyter: Tested on `JupyterLab 1.0.6`, `jupyter_core 4.5.0` and `IPython 6.4.0`
- Kernel: `conda_python3`
- Dependencies required
  - `sagemaker-python-sdk>=2.40.0`
    - Use `!pip install sagemaker==2.40.0` to download this dependency.
    - Kernel may need to be restarted after download.
- Expected Execution Role/permission
  - S3 access to the bucket that stores the notebook.

### Downloading Generated Modules
Download the generated data transformation modules and an SageMaker Autopilot helper module used by this notebook.
Those artifacts will be downloaded to **automl-woobin-artifacts** folder.

생성된 데이터 변환 모듈과 이 노트북에 사용되는 SageMaker 오토파일럿 도우미 모듈을 다운로드합니다.
이러한 아티팩트는 *automl-woobin-artifacts* 폴더로 다운로드됩니다.

In [1]:
!mkdir -p automl-woobin-artifacts
!aws s3 sync s3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin/sagemaker-automl-candidates/automl-woobin-pr-1-534756e62f4447ac91291d1363d87182be4ed7311617/generated_module automl-woobin-artifacts/generated_module --only-show-errors
!aws s3 sync s3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin/sagemaker-automl-candidates/automl-woobin-pr-1-534756e62f4447ac91291d1363d87182be4ed7311617/notebooks/sagemaker_automl automl-woobin-artifacts/sagemaker_automl --only-show-errors

import sys
sys.path.append("automl-woobin-artifacts")

### SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration

The following configuration has been derived from the SageMaker Autopilot job. These items configure where this notebook will
look for generated candidates, and where input and output data is stored on Amazon S3.

다음 구성은 SageMaker 자동 조종 작업에서 파생되었습니다. 이 항목은 이 노트북의 위치를 구성합니다
생성된 후보와 입력 및 출력 데이터가 Amazon S3에 저장된 위치를 찾습니다.

In [3]:
from sagemaker_automl import uid, AutoMLLocalRunConfig

# Where the preprocessed data from the existing AutoML job is stored
BASE_AUTOML_JOB_NAME = 'automl-woobin'
BASE_AUTOML_JOB_CONFIG = {
    'automl_job_name': BASE_AUTOML_JOB_NAME,
    'automl_output_s3_base_path': 's3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin',
    'data_transformer_image_repo_version': '2.5-1-cpu-py3',
    'algo_image_repo_versions': {'xgboost': '1.3-1-cpu-py3', 'linear-learner': 'training-cpu', 'mlp': 'training-cpu'},
    'algo_inference_image_repo_versions': {'xgboost': '1.3-1-cpu-py3', 'linear-learner': 'inference-cpu', 'mlp': 'inference-cpu'}
}

# Path conventions of the output data storage path from the local AutoML job run of this notebook
LOCAL_AUTOML_JOB_NAME = 'automl-woo-notebook-run-{}'.format(uid())
LOCAL_AUTOML_JOB_CONFIG = {
    'local_automl_job_name': LOCAL_AUTOML_JOB_NAME,
    'local_automl_job_output_s3_base_path': 's3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin/{}'.format(LOCAL_AUTOML_JOB_NAME),
    'data_processing_model_dir': 'data-processor-models',
    'data_processing_transformed_output_dir': 'transformed-data',
    'multi_algo_tuning_output_dir': 'multi-algo-tuning'
}

AUTOML_LOCAL_RUN_CONFIG = AutoMLLocalRunConfig(
    role='arn:aws:iam::938531851447:role/service-role/AmazonSageMaker-ExecutionRole-20230202T005614',
    base_automl_job_config=BASE_AUTOML_JOB_CONFIG,
    local_automl_job_config=LOCAL_AUTOML_JOB_CONFIG,
    security_config={'EnableInterContainerTrafficEncryption': False, 'VpcConfig': {}})

AUTOML_LOCAL_RUN_CONFIG.display()

This notebook is initialized to use the following configuration: 
        <table>
        <tr><th colspan=2>Name</th><th>Value</th></tr>
        <tr><th>General</th><th>Role</th><td>arn:aws:iam::938531851447:role/service-role/AmazonSageMaker-ExecutionRole-20230202T005614</td></tr>
        <tr><th rowspan=2>Base AutoML Job</th><th>Job Name</th><td>automl-woobin</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin</td></tr>
        <tr><th rowspan=5>Interactive Job</th><th>Job Name</th><td>automl-woo-notebook-run-15-14-44-38</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin/automl-woo-notebook-run-15-14-44-38</td></tr>
        <tr><th>Data Processing Trained Model Directory</th><td>s3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin/automl-woo-notebook-run-15-14-44-38/data-processor-models</td></tr>
        <tr><th>Data Processing Transformed Output</th><td>s3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin/automl-woo-notebook-run-15-14-44-38/transformed-data</td></tr>
        <tr><th>Algo Tuning Model Output Directory</th><td>s3://sagemaker-ap-northeast-2-938531851447/automl-house-price/output/automl-woobin/automl-woo-notebook-run-15-14-44-38/multi-algo-tuning</td></tr>
        </table>
        

## Candidate Pipelines

The `AutoMLLocalRunner` keeps track of selected candidates and automates many of the steps needed to execute feature engineering and tuning steps.

AutoMLlocalRunner는 선택된 후보를 추적하고 기능 엔지니어링 및 튜닝 단계를 실행하는 데 필요한 많은 단계를 자동화합니다.

In [5]:
from sagemaker_automl import AutoMLInteractiveRunner, AutoMLLocalCandidate

automl_interactive_runner = AutoMLInteractiveRunner(AUTOML_LOCAL_RUN_CONFIG)

### Generated Candidates

The SageMaker Autopilot Job has analyzed the dataset and has generated **10** machine learning
pipeline(s) that use **3** algorithm(s). Each pipeline contains a set of feature transformers and an
algorithm.

세이지메이커 오토파일럿 작업은 데이터 세트를 분석하고 3개의 알고리듬을 사용하는 10개의 기계 학습 파이프라인을 생성했다. 각 파이프라인에는 기능 변환기 세트와 알고리즘이 포함되어 있습니다.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. The resource configuration: instance type & count
1. Select candidate pipeline definitions by cells
1. The linked data transformation script can be reviewed and updated. Please refer to the [README.md](./automl-woobin-artifacts/generated_module/README.md) for detailed customization instructions.
</div>

**[dpp0-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp0.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [12]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp0",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2023-02-15 14:47:08,471 INFO root: Warning: pipeline candidate dpp0-xgboost has already been selected, replacing
2023-02-15 14:47:08,520 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2023-02-15 14:47:08,536 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2023-02-15 14:47:08,568 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp1-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp1.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [13]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp1",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "text/csv",
#         "sparse_encoding": False
#     },
#     "algorithm": {
#         "name": "xgboost",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#     }
# })

**[dpp2-linear-learner](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp2.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [14]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp2",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2023-02-15 14:47:12,497 INFO root: Warning: pipeline candidate dpp2-linear-learner has already been selected, replacing
2023-02-15 14:47:12,525 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2023-02-15 14:47:12,527 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2023-02-15 14:47:12,528 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: training-cpu.
2023-02-15 14:47:12,546 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2023-02-15 14:47:12,548 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2023-02-15 14:47:12,549 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: inference-cpu.
2023-02-15 14:47:12,568 INFO sagemaker.image_uris: Ignoring unnecessary instance ty

**[dpp3-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp3.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [ ]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp3",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "application/x-recordio-protobuf",
#         "sparse_encoding": True
#     },
#     "algorithm": {
#         "name": "xgboost",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#     }
# })

**[dpp4-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp4.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [ ]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp4",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "application/x-recordio-protobuf",
#         "sparse_encoding": True
#     },
#     "algorithm": {
#         "name": "xgboost",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#     }
# })

**[dpp5-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp5.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [ ]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp5",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "application/x-recordio-protobuf",
#         "sparse_encoding": True
#     },
#     "algorithm": {
#         "name": "xgboost",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#     }
# })

**[dpp6-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp6.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [ ]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp6",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "application/x-recordio-protobuf",
#         "sparse_encoding": True
#     },
#     "algorithm": {
#         "name": "xgboost",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#     }
# })

**[dpp7-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp7.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [ ]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp7",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "application/x-recordio-protobuf",
#         "sparse_encoding": True
#     },
#     "algorithm": {
#         "name": "xgboost",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#     }
# })

**[dpp8-xgboost](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp8.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [ ]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp8",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "application/x-recordio-protobuf",
#         "sparse_encoding": True
#     },
#     "algorithm": {
#         "name": "xgboost",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#     }
# })

**[dpp9-mlp](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp9.py)**: This data transformation strategy transforms 'categorical' features using [ThresholdOrdinalEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/feature_extraction/text.py). The
transformed data will be used to tune a *mlp* model. Here is the definition:

In [ ]:
# automl_interactive_runner.select_candidate({
#     "data_transformer": {
#         "name": "dpp9",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#             "volume_size_in_gb":  50
#         },
#         "transform_resource_config": {
#             "instance_type": "ml.m5.4xlarge",
#             "instance_count": 1,
#         },
#         "transforms_label": True,
#         "transformed_data_format": "text/csv",
#         "sparse_encoding": True
#     },
#     "algorithm": {
#         "name": "mlp",
#         "training_resource_config": {
#             "instance_type": "ml.m5.12xlarge",
#             "instance_count": 1,
#         },
#         "candidate_specific_static_hyperparameters": {
#             "num_categorical_features": '2',
#         }
#     }
# })

### Selected Candidates

You have selected the following candidates (please run the cell below and click on the feature transformer links for details):

In [15]:
automl_interactive_runner.display_candidates()

The feature engineering pipeline consists of two SageMaker jobs:

1. Generated trainable data transformer Python modules like [dpp0.py](automl-woobin-artifacts/generated_module/candidate_data_processors/dpp0.py), which has been downloaded to the local file system
2. A **training** job to train the data transformers
3. A **batch transform** job to apply the trained transformation to the dataset to generate the algorithm compatible data

The transformers and its training pipeline are built using open sourced **[sagemaker-scikit-learn-container][]** and **[sagemaker-scikit-learn-extension][]**.

[sagemaker-scikit-learn-container]: https://github.com/aws/sagemaker-scikit-learn-container
[sagemaker-scikit-learn-extension]: https://github.com/aws/sagemaker-scikit-learn-extension

## Executing the Candidate Pipelines

Each candidate pipeline consists of two steps, feature transformation and algorithm training.
For efficiency first execute the feature transformation step which will generate a featurized dataset on S3
for each pipeline.

After each featurized dataset is prepared, execute a multi-algorithm tuning job that will run tuning jobs
in parallel for each pipeline. This tuning job will execute training jobs to find the best set of
hyper-parameters for each pipeline, as well as finding the overall best performing pipeline.

### Run Data Transformation Steps

Now you are ready to start execution all data transformation steps.  The cell below may take some time to finish,
feel free to go grab a cup of coffee. To expedite the process you can set the number of `parallel_jobs` to be up to 10.
Please check the account limits to increase the limits before increasing the number of jobs to run in parallel.

각 후보 파이프라인은 기능 변환과 알고리듬 훈련이라는 두 단계로 구성된다. 효율성을 위해 먼저 각 파이프라인에 대해 S3에 피처화된 데이터 세트를 생성하는 피처 변환 단계를 실행한다.

각 기능화된 데이터 세트가 준비되면 각 파이프라인에 대해 튜닝 작업을 병렬로 실행하는 다중 알고리즘 튜닝 작업을 실행합니다. 이 튜닝 작업은 각 파이프라인에 대한 최적의 하이퍼 파라미터 세트를 찾을 뿐만 아니라 전체적으로 가장 성능이 좋은 파이프라인을 찾기 위해 교육 작업을 실행한다.

In [17]:
automl_interactive_runner.fit_data_transformers(parallel_jobs=1)

.2023-02-15 14:53:01,706 INFO root: [Worker_0:dpp0-xgboost]Executing step: train_data_transformer
2023-02-15 14:53:01,859 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2023-02-15 14:53:01,875 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2023-02-15 14:53:01,877 INFO sagemaker: Creating training-job with name: automl-woo-notebook-run-15-14-44-38-dpp0-train-15-14-52-54

2023-02-15 14:53:02 Starting - Starting the training job.......
2023-02-15 14:53:17 Starting - Preparing the instances for training...............
2023-02-15 14:54:01 Downloading - Downloading input data...........
2023-02-15 14:54:32 Training - Downloading the training image.
2023-02-15 14:54:37 Training - Training image download completed. Training in progress.........!
.
2023-02-15 14:55:08 Uploading - Uploading generated training model.
2023-02-15 14:55:18 Completed - Training job completed
2023-02-15 14:55:25,749 INFO root: [Worker_0:dpp0-xgboo

### Multi Algorithm Hyperparameter Tuning

Now that the algorithm compatible transformed datasets are ready, you can start the multi-algorithm model tuning job
to find the best predictive model. The following algorithm training job configuration for each
algorithm is auto-generated by the AutoML Job as part of the recommendation.

이제 알고리즘 호환 변환된 데이터 세트가 준비되었으므로 다중 알고리즘 모델 튜닝 작업을 시작하여 최적의 예측 모델을 찾을 수 있습니다. 각 알고리즘에 대한 다음 알고리즘 교육 작업 구성은 권장 사항의 일부로 AutoML Job에 의해 자동으로 생성됩니다.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Hyperparameter ranges
2. Objective metrics
3. Recommended static algorithm hyperparameters.

Please refers to [Xgboost tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) and [Linear learner tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner-tuning.html) for detailed explanations of the parameters.
</div>

The AutoML recommendation job has recommended the following hyperparameters, objectives and accuracy metrics for
the algorithm and problem type:

In [18]:
ALGORITHM_OBJECTIVE_METRICS = {
    'xgboost': 'validation:f1_binary',
    'linear-learner': 'validation:binary_f_beta',
    'mlp': 'validation:binary_f_beta',
}

STATIC_HYPERPARAMETERS = {
    'xgboost': {
        'objective': 'binary:logistic',
        'eval_metric': 'accuracy,f1_binary,auc,balanced_accuracy,precision,recall,logloss',
        'scale_pos_weight': 6.109333333333334,
        '_kfold': 5,
    },
    'linear-learner': {
        'predictor_type': 'binary_classifier',
        'ml_application': 'linear_learner',
        'loss_function': 'SoftmaxCrossEntropyLoss',
        'reporting_metrics': 'binary_classification_accuracy,binary_f_beta,roc_auc_score,balanced_accuracy,precision,recall,binary_logloss',
        'positive_example_weight_mult': 6.109333333333334,
        'eval_metric': 'binary_f_beta',
        'kfold': 5,
        'num_cv_rounds': 3,
        'prediction_storage_mode': 'store_cv_avg_predictions',
    },
    'mlp': {
        'problem_type': 'binary_classification',
        'positive_example_weight_mult': 6.109333333333334,
        'ml_application': 'mlp',
        'use_batchnorm': 'true',
        'activation': 'relu',
        'warmup_epochs': 10,
        'reporting_metrics': 'accuracy,binary_f_1,roc_auc,balanced_accuracy,precision,recall,logloss',
        'eval_metric': 'binary_f_1',
        'kfold': 5,
        'prediction_storage_mode': 'store_cv_avg_predictions',
    },
}

The following tunable hyperparameters search ranges are recommended for the Multi-Algo tuning job:

In [19]:
from sagemaker.parameter import CategoricalParameter, ContinuousParameter, IntegerParameter

ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES = {
    'xgboost': {
        'num_round': IntegerParameter(64, 1024, scaling_type='Logarithmic'),
        'max_depth': IntegerParameter(2, 8, scaling_type='Logarithmic'),
        'eta': ContinuousParameter(1e-3, 1.0, scaling_type='Logarithmic'),
        'gamma': ContinuousParameter(1e-6, 64.0, scaling_type='Logarithmic'),
        'min_child_weight': ContinuousParameter(1e-6, 32.0, scaling_type='Logarithmic'),
        'subsample': ContinuousParameter(0.5, 1.0, scaling_type='Linear'),
        'colsample_bytree': ContinuousParameter(0.3, 1.0, scaling_type='Linear'),
        'lambda': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
        'alpha': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
    },
    'linear-learner': {
        'mini_batch_size': IntegerParameter(128, 512, scaling_type='Linear'),
        'wd': ContinuousParameter(1e-12, 1e-2, scaling_type='Logarithmic'),
        'learning_rate': ContinuousParameter(1e-6, 1e-2, scaling_type='Logarithmic'),
    },
    'mlp': {
        'mini_batch_size': IntegerParameter(128, 512, scaling_type='Linear'),
        'learning_rate': ContinuousParameter(1e-6, 1e-2, scaling_type='Logarithmic'),
        'weight_decay': ContinuousParameter(1e-12, 1e-2, scaling_type='Logarithmic'),
        'dropout_prob': ContinuousParameter(0.25, 0.5, scaling_type='Linear'),
        'embedding_size_factor': ContinuousParameter(0.65, 0.95, scaling_type='Linear'),
        'network_type': CategoricalParameter(['feedforward', 'widedeep']),
        'layers': CategoricalParameter(['256', '50, 25', '100, 50', '200, 100', '256, 128', '300, 150', '200, 100, 50']),
    },
}

#### Prepare Multi-Algorithm Tuner Input

To use the multi-algorithm HPO tuner, prepare some inputs and parameters. Prepare a dictionary whose key is the name of the trained pipeline candidates and the values are respectively:

1. Estimators for the recommended algorithm
2. Hyperparameters search ranges
3. Objective metrics

In [20]:
multi_algo_tuning_parameters = automl_interactive_runner.prepare_multi_algo_parameters(
    objective_metrics=ALGORITHM_OBJECTIVE_METRICS,
    static_hyperparameters=STATIC_HYPERPARAMETERS,
    hyperparameters_search_ranges=ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES)

Below you prepare the inputs data to the multi-algo tuner:

In [21]:
multi_algo_tuning_inputs = automl_interactive_runner.prepare_multi_algo_inputs()

#### Create Multi-Algorithm Tuner

With the recommended Hyperparameter ranges and the transformed dataset, create a multi-algorithm model tuning job
that coordinates hyper parameter optimizations across the different possible algorithms and feature processing strategies.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Tuner strategy: [Bayesian](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization), [Random Search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)
2. Objective type: `Minimize`, `Maximize`, see [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization)
3. Max Job size: the max number of training jobs HPO would be launching to run experiments. Note the default value is **250**
    which is the default of the managed flow.
4. Parallelism. Number of jobs that will be executed in parallel. Higher value will expedite the tuning process.
    Please check the account limits to increase the limits before increasing the number of jobs to run in parallel
5. Please use a different tuning job name if you re-run this cell after applied customizations.
</div>

튜너 전략: 베이지안, 랜덤 검색
목표 유형: 최소화, 최대화, 최적화 참조
최대 작업 크기: HPO가 실험을 실행하기 위해 실행할 최대 교육 작업 수입니다. 기본값은 관리되는 흐름의 기본값인 250입니다.
평행성. 병렬로 실행될 작업 수입니다. 값이 클수록 튜닝 프로세스가 빨라집니다. 병렬로 실행할 작업 수를 늘리기 전에 계정 제한을 확인하여 제한을 늘리십시오
사용자 지정을 적용한 후 이 셀을 다시 실행하는 경우 다른 튜닝 작업 이름을 사용하십시오.

In [22]:
from sagemaker.tuner import HyperparameterTuner

base_tuning_job_name = "{}-tuning".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name)

tuner = HyperparameterTuner.create(
    base_tuning_job_name=base_tuning_job_name,
    strategy='Bayesian',
    objective_type='Maximize',
    max_parallel_jobs=1, #4
    max_jobs=8, #250
    **multi_algo_tuning_parameters,
)

#### Run Multi-Algorithm Tuning

Now you are ready to start running the **Multi-Algo Tuning** job. After the job is finished, store the tuning job name which you use to select models in the next section.
The tuning process will take some time, please track the progress in the Amazon SageMaker Hyperparameter tuning jobs console.

In [23]:
from IPython.display import display, Markdown

# Run tuning
tuner.fit(inputs=multi_algo_tuning_inputs, include_cls_metadata=None)
tuning_job_name = tuner.latest_tuning_job.name

display(
    Markdown(f"Tuning Job {tuning_job_name} started, please track the progress from [here](https://{AUTOML_LOCAL_RUN_CONFIG.region}.console.aws.amazon.com/sagemaker/home?region={AUTOML_LOCAL_RUN_CONFIG.region}#/hyper-tuning-jobs/{tuning_job_name})"))

# Wait for tuning job to finish
tuner.wait()

2023-02-15 15:12:00,397 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2023-02-15 15:12:00,426 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2023-02-15 15:12:00,429 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2023-02-15 15:12:00,448 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2023-02-15 15:12:00,449 WARNING sagemaker.estimator: No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
2023-02-15 15:12:00,455 INFO sagemaker: Creating hyperparameter tuning job with name: automl-woo-notebook--230215-1512
.......................................................................................................................................................................................................................................................................

Tuning Job automl-woo-notebook--230215-1512 started, please track the progress from [here](https://ap-northeast-2.console.aws.amazon.com/sagemaker/home?region=ap-northeast-2#/hyper-tuning-jobs/automl-woo-notebook--230215-1512)

!


## Model Selection and Deployment

This section guides you through the model selection process. Afterward, you construct an inference pipeline
on Amazon SageMaker to host the best candidate.

Because you executed the feature transformation and algorithm training in two separate steps, you now need to manually
link each trained model with the feature transformer that it is associated with. When running a regular Amazon
SageMaker Autopilot job, this will automatically be done for you.

### Tuning Job Result Overview

The performance of each candidate pipeline can be viewed as a Pandas dataframe. For more interactive usage please
refers to [model tuning monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html).

In [24]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

SAGEMAKER_SESSION = AUTOML_LOCAL_RUN_CONFIG.sagemaker_session
SAGEMAKER_ROLE = AUTOML_LOCAL_RUN_CONFIG.role

tuner_analytics = HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

learning_rate  mini_batch_size        wd  \
7            NaN              NaN       NaN   
3            NaN              NaN       NaN   
5            NaN              NaN       NaN   
2            NaN              NaN       NaN   
0       0.009778            391.0  0.004683   
1       0.003217            271.0  0.000002   
6       0.002460            503.0  0.000041   
4       0.000299            478.0  0.000002   

                                 TrainingJobName TrainingJobStatus  \
7  automl-woo-notebook--230215-1512-001-7db457db         Completed   
3  automl-woo-notebook--230215-1512-005-657df1b4         Completed   
5  automl-woo-notebook--230215-1512-003-1ee687a7         Completed   
2  automl-woo-notebook--230215-1512-006-378d72d3         Completed   
0  automl-woo-notebook--230215-1512-008-dc7782f7         Completed   
1  automl-woo-notebook--230215-1512-007-f78f06f2         Completed   
6  automl-woo-notebook--230215-1512-002-f343e535         Completed   
4  automl-woo-notebook--230215-1512-004-6945c16e         Completed   

   FinalObjectiveValue         TrainingStartTime           TrainingEndTime  \
7             0.793990 2023-02-15 15:13:16+00:00 2023-02-15 15:14:54+00:00   
3             0.779480 2023-02-15 15:24:04+00:00 2023-02-15 15:25:51+00:00   
5             0.769810 2023-02-15 15:18:59+00:00 2023-02-15 15:20:31+00:00   
2             0.683780 2023-02-15 15:26:01+00:00 2023-02-15 15:29:10+00:00   
0             0.458150 2023-02-15 15:33:36+00:00 2023-02-15 15:34:38+00:00   
1             0.448669 2023-02-15 15:29:44+00:00 2023-02-15 15:32:37+00:00   
6             0.418440 2023-02-15 15:15:21+00:00 2023-02-15 15:18:34+00:00   
4             0.247788 2023-02-15 15:20:55+00:00 2023-02-15 15:23:37+00:00   

   TrainingElapsedTimeSeconds TrainingJobDefinitionName     alpha  \
7                        98.0              dpp0-xgboost  0.588088   
3                       107.0              dpp0-xgboost  0.000949   
5                        92.0              dpp0-xgboost  1.315062   
2                       189.0              dpp0-xgboost  0.000004   
0                        62.0       dpp2-linear-learner       NaN   
1                       173.0       dpp2-linear-learner       NaN   
6                       193.0       dpp2-linear-learner       NaN   
4                       162.0       dpp2-linear-learner       NaN   

   colsample_bytree       eta      gamma    lambda  max_depth  \
7          0.391735  0.144569   0.027247  1.195854        4.0   
3          0.562214  0.001375   0.000168  1.402580        7.0   
5          0.466512  0.710914   0.000007  0.166086        3.0   
2          0.865630  0.035534  23.241531  0.000001        2.0   
0               NaN       NaN        NaN       NaN        NaN   
1               NaN       NaN        NaN       NaN        NaN   
6               NaN       NaN        NaN       NaN        NaN   
4               NaN       NaN        NaN       NaN        NaN   

   min_child_weight  num_round  subsample  
7          0.011248      125.0   0.896994  
3          0.002304      118.0   0.606925  
5          0.139939       66.0   0.997384  
2          0.000032      748.0   0.889711  
0               NaN        NaN        NaN  
1               NaN        NaN        NaN  
6               NaN        NaN        NaN  
4               NaN        NaN        NaN

The best training job can be selected as below:

<div class="alert alert-info"> 💡 <strong>Tips: </strong>
You could select alternative job by using the value from `TrainingJobName` column above and assign to `best_training_job` below
</div>

In [25]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)
best_training_job = attached_tuner.best_training_job()

print("Best Multi Algorithm HPO training job name is {}".format(best_training_job))

Best Multi Algorithm HPO training job name is automl-woo-notebook--230215-1512-001-7db457db


### Linking Best Training Job with Feature Pipelines

Finally, deploy the best training job to Amazon SageMaker along with its companion feature engineering models.
At the end of the section, you get an endpoint that's ready to serve online inference or start batch transform jobs!

기능 파이프라인과 최상의 교육 작업 연결
마지막으로 Amazon SageMaker의 동반 기능 엔지니어링 모델과 함께 최상의 교육 작업을 배포합니다. 섹션이 끝나면 온라인 추론을 제공하거나 일괄 변환 작업을 시작할 준비가 된 엔드포인트가 제공됩니다!

Deploy a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/pipeline.html) that has multiple containers of the following:

1. Data Transformation Container: a container built from the model we selected and trained during the data transformer sections
2. Algorithm Container: a container built from the trained model we selected above from the best HPO training job.
3. Inverse Label Transformer Container: a container that converts numerical intermediate prediction value back to non-numerical label value.

Get both best data transformation model and algorithm model from best training job and create an pipeline model:

다음과 같은 여러 컨테이너가 있는 Pipeline Model을 배포합니다:

데이터 변환 컨테이너: 데이터 변환 섹션에서 선택하고 교육한 모델을 기반으로 구축된 컨테이너
Algorithm Container: 최상의 HPO 교육 작업에서 위에서 선택한 훈련된 모델을 기반으로 제작된 컨테이너입니다.
Inverse Label Transformer Container: 숫자 중간 예측 값을 다시 숫자가 아닌 레이블 값으로 변환하는 컨테이너입니다.
최고의 교육 작업을 통해 최고의 데이터 변환 모델과 알고리즘 모델을 모두 확보하고 파이프라인 모델을 생성하십시오:

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel
from sagemaker_automl import select_inference_output

# Get a data transformation model from chosen candidate
best_candidate = automl_interactive_runner.choose_candidate(df_tuning_job_analytics, best_training_job)
best_data_transformer_model = best_candidate.get_data_transformer_model(role=SAGEMAKER_ROLE, sagemaker_session=SAGEMAKER_SESSION)

# Our first data transformation container will always return recordio-protobuf format
best_data_transformer_model.env["SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT"] = 'application/x-recordio-protobuf'
# Add environment variable for sparse encoding
if best_candidate.data_transformer_step.sparse_encoding:
    best_data_transformer_model.env["AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF"] = '1'

# Get a algo model from chosen training job of the candidate
algo_estimator = Estimator.attach(best_training_job)
best_algo_model = algo_estimator.create_model(**best_candidate.algo_step.get_inference_container_config())

# Final pipeline model is composed of data transformation models and algo model and an
# inverse label transform model if we need to transform the intermediates back to non-numerical value
model_containers = [best_data_transformer_model, best_algo_model]
if best_candidate.transforms_label:
    model_containers.append(best_candidate.get_data_transformer_model(
        transform_mode="inverse-label-transform",
        role=SAGEMAKER_ROLE,
        sagemaker_session=SAGEMAKER_SESSION))

# This model can emit response ['predicted_label', 'probability', 'labels', 'probabilities']. To enable the model to emit one or more
# of the response content, pass the keys to `output_key` keyword argument in the select_inference_output method.

model_containers = select_inference_output("BinaryClassification", model_containers, output_keys=['predicted_label'])


pipeline_model = PipelineModel(
    name="AutoML-{}".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name),
    role=SAGEMAKER_ROLE,
    models=model_containers,
    vpc_config=AUTOML_LOCAL_RUN_CONFIG.vpc_config)

### Deploying Best Pipeline

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. You can customize the initial instance count and instance type used to deploy this model.
2. Endpoint name can be changed to avoid conflict with existing endpoints.

</div>

Finally, deploy the model to SageMaker to make it functional.

In [ ]:
pipeline_model.deploy(initial_instance_count=1,
                      instance_type='ml.m5.2xlarge',
                      endpoint_name=pipeline_model.name,
                      wait=True)

Congratulations! Now you could visit the sagemaker
[endpoint console page](https://ap-northeast-2.console.aws.amazon.com/sagemaker/home?region=ap-northeast-2#/endpoints) to find the deployed endpoint (it'll take a few minutes to be in service).

<div class="alert alert-warning">
    <strong>To rerun this notebook, delete or change the name of your endpoint!</strong> <br>
If you rerun this notebook, you'll run into an error on the last step because the endpoint already exists. You can either delete the endpoint from the endpoint console page or you can change the <code>endpoint_name</code> in the previous code block.
</div>